In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
FILENAME = '/Users/shanewang/Desktop/anran/May.xlsx'
df = pd.read_excel(FILENAME, sheet_name='到项目') #一级考核-错号率
raw = df.copy(deep=True)
raw.head()

,行标签,A1题甄别失败,S2题甄别失败,S4a题甄别失败,S4b题甄别失败,S6a题甄别失败,S6题甄别失败,S7题甄别失败,成功,关机/占线/无法接通/语音信箱,过滤问卷拒访,开场白拒访,空号/号码过期,磨合期-延期交付（未交付）,审核作废,时间不宜,停机,未购买此楼盘,未接触,无此人/打错了,无人接听,语言不通,直接挂断/被访者不说话,重复访问,主体问卷拒访,准业主-提前交付（已交付）,总计,接触量,错号量,接通量,错号率
0,集团,45.0,16.0,82.0,32.0,62.0,31.0,80.0,6046.0,2118.0,94.0,1378.0,151.0,1.0,4.0,1712.0,121.0,92.0,24702.0,66.0,8406.0,36.0,432.0,19.0,454.0,12.0,46192,21490,446,10966,0.020754
1,上海区域,6.0,3.0,13.0,9.0,3.0,5.0,7.0,788.0,352.0,13.0,217.0,24.0,NaN,NaN,277.0,16.0,35.0,2926.0,19.0,1322.0,2.0,64.0,1.0,65.0,2.0,6169,3243,97,1569,0.029911
2,蒙沪公司,NaN,NaN,NaN,1.0,NaN,NaN,NaN,30.0,10.0,1.0,9.0,NaN,NaN,NaN,12.0,NaN,NaN,137.0,1.0,53.0,NaN,1.0,NaN,3.0,NaN,258,121,1,58,0.008264
3,呼和浩特瀚海壹號,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,16.0,NaN,8.0,NaN,NaN,NaN,1.0,NaN,35,19,0,9,0.000000
4,准业主,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,16.0,NaN,8.0,NaN,NaN,NaN,1.0,NaN,35,19,0,9,0.000000


In [3]:
def simplify(raw: pd.DataFrame):    
    labels = ['行标签', '总计', '接触量', '接触数','错号数', '错号量']
    for name in raw.columns.tolist():
        if name == '错号数':
            raw.rename(columns={'错号数':'错号量'}, inplace=True)
        if name == '接触数':
            raw.rename(columns={'接触数':'接触量'}, inplace=True)
        if name not in labels:
            raw.drop(name, axis=1, inplace=True)

        
simplify(raw)

raw.head(10)

,行标签,总计,接触量,错号量
0,集团,46192,21490,446
1,上海区域,6169,3243,97
2,蒙沪公司,258,121,1
3,呼和浩特瀚海壹號,35,19,0
4,准业主,35,19,0
5,1-准业主1,35,19,0
6,呼和浩特玉兰天宸,196,75,0
7,准业主,196,75,0
8,1-准业主1,25,17,0
9,2-准业主2,171,58,0


In [4]:
# raw.to_excel('sampleTest.xlsx', index=False)

In [5]:
city_names = []
deprecated_names = ['集团', '上海区域']
# deprecated_names = ['集团']
for name in raw['行标签'].tolist():
    if name not in deprecated_names and '准业主' not in name \
                                    and '磨合期' not in name \
                                    and '老业主' not in name \
                                    and '稳定期' not in name:
        city_names.append(name)

print(city_names)
len(city_names)

['蒙沪公司', '呼和浩特瀚海壹號', '呼和浩特玉兰天宸', '上海桃花源', '南京公司', '滁州凯迪融创玉兰公馆', '滁州凯迪融创御园', '南京玖溪桃花源', '南京融创中南御园', '南京汤山颐和府', '南京玉兰公馆', '南通公司', '南通江语海', '南通九里桃源', '南通玖樟台', '南通时代壹号', '南通玉兰公馆', '启东海上桃源', '启东桃源里', '商置公司', '上海香溢花城', '上海公司', '上海葛洲坝虹桥玫瑰公馆', '上海领馆壹号院', '上海玫瑰公馆', '上海山水拾间', '苏北公司', '淮安融创广场', '淮南玉兰公馆', '徐州都会星宸', '徐州枫丹公馆', '徐州凤鸣桃源', '徐州凤鸣桃源风雅颂', '徐州淮海壹号', '枣庄御园', '苏南公司', '常州滨湖御园', '常州御园一二三期', '江阴江南御园', '江阴敔山桃源', '靖江玉兰公馆', '苏州泰兴御园', '泰兴桃源府南苑', '泰州融创玉兰花园', '无锡瑷颐湾', '无锡大塘御园', '无锡东方御园', '无锡枫丹御园', '无锡惠山国际社区', '无锡惠山映', '无锡江阴玉兰公馆', '无锡玖里映月', '无锡理想城市', '无锡森邻森邻', '无锡文旅城', '无锡西羲里', '无锡香樟园', '无锡壹号院', '无锡玉兰公馆', '无锡运河壹号府', '无锡长江映', '苏中公司', '芜湖玉兰公馆', '扬州华府天地', '镇江观澜公馆', '镇江御园', '苏州公司', '昆山玉兰公馆', '苏州泊云庭', '苏州浦西玫瑰园', '苏州壹号院', '张家港滨江御园', '新疆公司', '乌鲁木齐江南桃源']


74

In [6]:
row_records = []
for _, row in raw.iterrows():
    try:
        row_records.append([row['行标签'], row['接触量'], row['错号量']])
    except Exception as e:
        print(e)
#         row_records.append([row['行标签'], row['总计'], row['错号量']])
    # print(row['行标签'], row['总计'], row['错号数'])
row_records

[['集团', 21490, 446],
 ['上海区域', 3243, 97],
 ['蒙沪公司', 121, 1],
 ['呼和浩特瀚海壹號', 19, 0],
 ['准业主', 19, 0],
 ['1-准业主1', 19, 0],
 ['呼和浩特玉兰天宸', 75, 0],
 ['准业主', 75, 0],
 ['1-准业主1', 17, 0],
 ['2-准业主2', 58, 0],
 ['上海桃花源', 27, 1],
 ['磨合期', 6, 1],
 ['3-磨合期1', 6, 1],
 ['准业主', 21, 0],
 ['1-准业主1', 3, 0],
 ['2-准业主2', 18, 0],
 ['南京公司', 207, 2],
 ['滁州凯迪融创玉兰公馆', 51, 0],
 ['准业主', 51, 0],
 ['1-准业主1', 19, 0],
 ['2-准业主2', 32, 0],
 ['滁州凯迪融创御园', 20, 1],
 ['准业主', 20, 1],
 ['1-准业主1', 6, 0],
 ['2-准业主2', 14, 1],
 ['南京玖溪桃花源', 54, 0],
 ['磨合期', 54, 0],
 ['3-磨合期1', 21, 0],
 ['4-磨合期2', 33, 0],
 ['南京融创中南御园', 65, 1],
 ['磨合期', 49, 1],
 ['3-磨合期1', 49, 1],
 ['准业主', 16, 0],
 ['1-准业主1', 6, 0],
 ['2-准业主2', 10, 0],
 ['南京汤山颐和府', 12, 0],
 ['准业主', 12, 0],
 ['1-准业主1', 12, 0],
 ['南京玉兰公馆', 5, 0],
 ['磨合期', 5, 0],
 ['3-磨合期1', 5, 0],
 ['南通公司', 423, 9],
 ['南通江语海', 75, 1],
 ['准业主', 75, 1],
 ['1-准业主1', 75, 1],
 ['南通九里桃源', 189, 3],
 ['准业主', 189, 3],
 ['1-准业主1', 2, 0],
 ['2-准业主2', 187, 3],
 ['南通玖樟台', 79, 2],
 ['准业主', 79, 2],
 ['1-准业主1', 29, 0]

In [7]:
wrong2city = {}
city_appeared = []
cities = []
slice_idx = []
zyz = []
mhq = []
wdq = []
lyz = []

for i in range(0, len(row_records)):
    tag = row_records[i][0]
    if tag in city_names:
        wrong2city[row_records[i][2]] = tag
        city_appeared.append(tag)
        slice_idx.append(i)
slice_idx.append(len(row_records))
for i in range(len(slice_idx)-1):
    cities.append(row_records[slice_idx[i]:slice_idx[i+1]])
print(cities[0])    
print(city_appeared)
print(slice_idx)

[['蒙沪公司', 121, 1]]
['蒙沪公司', '呼和浩特瀚海壹號', '呼和浩特玉兰天宸', '上海桃花源', '南京公司', '滁州凯迪融创玉兰公馆', '滁州凯迪融创御园', '南京玖溪桃花源', '南京融创中南御园', '南京汤山颐和府', '南京玉兰公馆', '南通公司', '南通江语海', '南通九里桃源', '南通玖樟台', '南通时代壹号', '南通玉兰公馆', '启东海上桃源', '启东桃源里', '商置公司', '上海香溢花城', '上海公司', '上海葛洲坝虹桥玫瑰公馆', '上海领馆壹号院', '上海玫瑰公馆', '上海山水拾间', '苏北公司', '淮安融创广场', '淮南玉兰公馆', '徐州都会星宸', '徐州枫丹公馆', '徐州凤鸣桃源', '徐州凤鸣桃源风雅颂', '徐州淮海壹号', '枣庄御园', '苏南公司', '常州滨湖御园', '常州御园一二三期', '江阴江南御园', '江阴敔山桃源', '靖江玉兰公馆', '苏州泰兴御园', '泰兴桃源府南苑', '泰州融创玉兰花园', '无锡瑷颐湾', '无锡大塘御园', '无锡东方御园', '无锡枫丹御园', '无锡惠山国际社区', '无锡惠山映', '无锡江阴玉兰公馆', '无锡玖里映月', '无锡理想城市', '无锡森邻森邻', '无锡文旅城', '无锡西羲里', '无锡香樟园', '无锡壹号院', '无锡玉兰公馆', '无锡运河壹号府', '无锡长江映', '苏中公司', '芜湖玉兰公馆', '扬州华府天地', '镇江观澜公馆', '镇江御园', '苏州公司', '昆山玉兰公馆', '苏州泊云庭', '苏州浦西玫瑰园', '苏州壹号院', '张家港滨江御园', '新疆公司', '乌鲁木齐江南桃源']
[2, 3, 6, 10, 16, 17, 21, 25, 29, 35, 38, 41, 42, 45, 49, 53, 56, 59, 63, 67, 68, 73, 74, 78, 81, 85, 89, 90, 94, 97, 100, 104, 109, 112, 115, 119, 120, 124, 127, 131, 134, 138, 142, 146, 150, 153, 157, 161, 165, 169, 173, 176, 179, 183, 18

In [8]:
for city_info in cities:
    CITY = city_info[0][0]
    for line in city_info[1:]:
        if '准业主' in line[0] and '准业主' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            zyz.append(line)
        if '磨合期' in line[0] and '磨合期' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            mhq.append(line)
        if '稳定期' in line[0] and '稳定期' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            wdq.append(line)
        if '老业主' in line[0] and '老业主' != line[0].strip():
            line[0] = CITY + '-' + line[0]
            lyz.append(line)
zyz

[['呼和浩特瀚海壹號-1-准业主1', 19, 0],
 ['呼和浩特玉兰天宸-1-准业主1', 17, 0],
 ['呼和浩特玉兰天宸-2-准业主2', 58, 0],
 ['上海桃花源-1-准业主1', 3, 0],
 ['上海桃花源-2-准业主2', 18, 0],
 ['滁州凯迪融创玉兰公馆-1-准业主1', 19, 0],
 ['滁州凯迪融创玉兰公馆-2-准业主2', 32, 0],
 ['滁州凯迪融创御园-1-准业主1', 6, 0],
 ['滁州凯迪融创御园-2-准业主2', 14, 1],
 ['南京融创中南御园-1-准业主1', 6, 0],
 ['南京融创中南御园-2-准业主2', 10, 0],
 ['南京汤山颐和府-1-准业主1', 12, 0],
 ['南通江语海-1-准业主1', 75, 1],
 ['南通九里桃源-1-准业主1', 2, 0],
 ['南通九里桃源-2-准业主2', 187, 3],
 ['南通玖樟台-1-准业主1', 29, 0],
 ['南通玖樟台-2-准业主2', 50, 2],
 ['南通时代壹号-1-准业主1', 44, 1],
 ['启东海上桃源-1-准业主1', 3, 0],
 ['启东海上桃源-2-准业主2', 4, 0],
 ['启东桃源里-1-准业主1', 15, 2],
 ['启东桃源里-2-准业主2', 8, 0],
 ['上海香溢花城-1-准业主1', 2, 0],
 ['上海葛洲坝虹桥玫瑰公馆-1-准业主1', 3, 0],
 ['上海葛洲坝虹桥玫瑰公馆-2-准业主2', 25, 0],
 ['上海山水拾间-1-准业主1', 3, 0],
 ['上海山水拾间-2-准业主2', 28, 0],
 ['淮安融创广场-1-准业主1', 56, 1],
 ['淮安融创广场-2-准业主2', 172, 3],
 ['淮南玉兰公馆-1-准业主1', 15, 2],
 ['徐州都会星宸-1-准业主1', 69, 0],
 ['徐州枫丹公馆-1-准业主1', 57, 0],
 ['徐州枫丹公馆-2-准业主2', 24, 0],
 ['徐州凤鸣桃源-1-准业主1', 1, 0],
 ['徐州凤鸣桃源风雅颂-2-准业主2', 9, 0],
 ['徐州淮海壹号-2-准业主2', 23, 1],
 ['枣庄御园-1

In [9]:
"""Get city info
    :return format: {CITY: [total, wrong]}
"""
def analyze(l: list):
    res = {}
    for i in range(len(l)):
        city = l[i][0].split('-')[0]
        if not res.__contains__(city):
            res[city] = [l[i][1], l[i][2]]
        else:
            res[city][0] += l[i][1]
            res[city][1] += l[i][2]
    return res

"""Calculate wrong rate
    :return format: {CITY: [total, wrong, wrong rate]}
"""
def calculate(d: dict):
    for k in d.keys():
        print(k, d[k])
        d[k].append(float('{:.5f}'.format(d[k][1]/d[k][0])))
    return d

In [10]:
zyz_list = []
mhq_list = []
wdq_list = []
lyz_list = []
for k, v in calculate(analyze(zyz)).items():
    zyz_list.append(tuple([k, '准业主'] + v))
for k, v in calculate(analyze(mhq)).items():
    mhq_list.append(tuple([k, '磨合期'] + v))
for k, v in calculate(analyze(wdq)).items():
    wdq_list.append(tuple([k, '稳定期'] + v))
for k, v in calculate(analyze(lyz)).items():
    lyz_list.append(tuple([k, '老业主'] + v))

呼和浩特瀚海壹號 [19, 0]
呼和浩特玉兰天宸 [75, 0]
上海桃花源 [21, 0]
滁州凯迪融创玉兰公馆 [51, 0]
滁州凯迪融创御园 [20, 1]
南京融创中南御园 [16, 0]
南京汤山颐和府 [12, 0]
南通江语海 [75, 1]
南通九里桃源 [189, 3]
南通玖樟台 [79, 2]
南通时代壹号 [44, 1]
启东海上桃源 [7, 0]
启东桃源里 [23, 2]
上海香溢花城 [2, 0]
上海葛洲坝虹桥玫瑰公馆 [28, 0]
上海山水拾间 [31, 0]
淮安融创广场 [228, 4]
淮南玉兰公馆 [15, 2]
徐州都会星宸 [69, 0]
徐州枫丹公馆 [81, 0]
徐州凤鸣桃源 [1, 0]
徐州凤鸣桃源风雅颂 [9, 0]
徐州淮海壹号 [23, 1]
枣庄御园 [13, 0]
常州滨湖御园 [95, 3]
江阴敔山桃源 [23, 0]
靖江玉兰公馆 [136, 1]
苏州泰兴御园 [48, 1]
泰兴桃源府南苑 [6, 0]
泰州融创玉兰花园 [40, 2]
无锡大塘御园 [168, 6]
无锡枫丹御园 [6, 0]
无锡惠山国际社区 [8, 0]
无锡惠山映 [66, 0]
无锡玖里映月 [6, 0]
无锡文旅城 [13, 0]
无锡西羲里 [109, 4]
无锡香樟园 [1, 0]
无锡壹号院 [22, 1]
无锡运河壹号府 [81, 1]
无锡长江映 [6, 0]
镇江观澜公馆 [47, 1]
镇江御园 [33, 1]
昆山玉兰公馆 [86, 3]
苏州泊云庭 [5, 1]
苏州浦西玫瑰园 [49, 0]
乌鲁木齐江南桃源 [32, 0]
上海桃花源 [6, 1]
南京玖溪桃花源 [54, 0]
南京融创中南御园 [49, 1]
南京玉兰公馆 [5, 0]
南通玉兰公馆 [6, 0]
上海香溢花城 [1, 0]
上海领馆壹号院 [40, 0]
上海玫瑰公馆 [221, 35]
徐州凤鸣桃源 [1, 0]
常州御园一二三期 [16, 0]
江阴江南御园 [63, 1]
无锡瑷颐湾 [23, 0]
无锡东方御园 [158, 4]
无锡江阴玉兰公馆 [7, 0]
无锡理想城市 [3, 0]
无锡森邻森邻 [2, 0]
无锡香樟园 [26, 1]
无锡玉兰公馆 [50, 1]
无锡运河壹号府 [47, 4]


In [11]:
labels = ['城市', '业主阶段', '接触量', '错号量', '错号率']
zyz_xlxs = pd.DataFrame(data=zyz_list, columns=labels)
zyz_xlxs = zyz_xlxs.sort_values(by='错号率', ascending=False)
zyz_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率
44,苏州泊云庭,准业主,5,1,0.20000
17,淮南玉兰公馆,准业主,15,2,0.13333
12,启东桃源里,准业主,23,2,0.08696
29,泰州融创玉兰花园,准业主,40,2,0.05000
4,滁州凯迪融创御园,准业主,20,1,0.05000
38,无锡壹号院,准业主,22,1,0.04545
22,徐州淮海壹号,准业主,23,1,0.04348
36,无锡西羲里,准业主,109,4,0.03670
30,无锡大塘御园,准业主,168,6,0.03571
43,昆山玉兰公馆,准业主,86,3,0.03488


In [12]:
mhq_xlxs = pd.DataFrame(data=mhq_list, columns=labels)
mhq_xlxs = mhq_xlxs.sort_values(by='错号率', ascending=False)
mhq_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率
0,上海桃花源,磨合期,6,1,0.16667
7,上海玫瑰公馆,磨合期,221,35,0.15837
23,苏州壹号院,磨合期,7,1,0.14286
18,无锡运河壹号府,磨合期,47,4,0.08511
19,芜湖玉兰公馆,磨合期,70,3,0.04286
16,无锡香樟园,磨合期,26,1,0.03846
12,无锡东方御园,磨合期,158,4,0.02532
20,扬州华府天地,磨合期,129,3,0.02326
2,南京融创中南御园,磨合期,49,1,0.02041
17,无锡玉兰公馆,磨合期,50,1,0.02000


In [13]:
wdq_xlxs = pd.DataFrame(data=wdq_list, columns=labels)
wdq_xlxs = wdq_xlxs.sort_values(by='错号率', ascending=False)
wdq_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率


In [14]:
lyz_xlxs = pd.DataFrame(data=lyz_list, columns=labels)
lyz_xlxs = lyz_xlxs.sort_values(by='错号率', ascending=False)
lyz_xlxs.head(20)

,城市,业主阶段,接触量,错号量,错号率


In [15]:
# Shanghai
def getShanghaiInfo(raw, zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs):
    for _, rows in raw.iterrows():
        if '准业主' in rows['行标签']:
            # tuple = ('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量'])))
            # print(('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
            shzyz = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['准业主'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            zyz_xlxs = pd.concat([zyz_xlxs, shzyz])

        if '磨合期' in rows['行标签']:
            shmhq = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['磨合期'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})

            mhq_xlxs = pd.concat([mhq_xlxs, shmhq])
        if '稳定期' in rows['行标签']:
            shwdq = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['稳定期'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            wdq_xlxs = pd.concat([wdq_xlxs, shwdq])
        if '老业主' in rows['行标签']:
            shlyz = pd.DataFrame({'城市': ['上海区域'],
                                 '业主阶段': ['老业主'],
                                 '接触量': rows['接触量'],
                                 '错号数': rows['错号量'],
                                 '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
            lyz_xlxs = pd.concat([lyz_xlxs, shlyz])
    return [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]

dflist = [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]
try:
    df = pd.read_excel(FILENAME, sheet_name='上海区域')
    raw = df.copy(deep=True)
    simplify(raw)
    dflist = getShanghaiInfo(raw, zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs)
except Exception as e:
    print(e)
    # print("No such sheet!")
    pass

No sheet named <'上海区域'>


In [27]:
# Excel weiter funtion
def multiple_dfs(df_list, sheets, file_name, spaces):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    row = 0
    for dataframe in df_list:
        dataframe.to_excel(writer, sheet_name=sheets, startrow=row, startcol=0, index=False)   
        row = row + len(dataframe.index) + spaces + 1
    writer.save()

# write list of dataframes
import time
file_name = FILENAME.strip().split('/')[-1].strip().split('.')[0]+'-'+time.strftime("%Y%m%d", time.localtime())+'.xlsx'
multiple_dfs(dflist, 'Validation', file_name, 1)
print("File processed completed!")

File processed completed!
